This code was used in "Predict The News Category Hackathon" ( https://machinehack.com/hackathon/predict_the_news_category_hackathon/overview) and resulted in public score of 0.9829 - 0.0094 behind the winning submission (rank 53 as of 15.01.2022).

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
from build_model import build_model
from preprocessing import create_tokenizer_from_hub_module, convert_single_example, \
                        convert_examples_to_features, convert_text_to_examples, initialize_vars

In [4]:
from plot_functions import plot_label

In [5]:
sess = tf.compat.v1.Session()

Let's load the data and see what our columns look like.
Note that there is one column with prediction target and it is non-binary. This means that we are dealing with multi-class classification.

In [6]:
data = pd.read_excel(r'Data_Train.xlsx')
data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3.0
1,How formidable is the opposition alliance amon...,0.0
2,Most Asian currencies were trading lower today...,3.0
3,"If you want to answer any question, click on ‘...",1.0
4,"In global markets, gold prices edged up today ...",3.0


In [7]:
data.shape

(7628, 2)

Using 0.1 as dev size

In [8]:
train, dev = train_test_split(data, test_size=0.1)

In [9]:
train.shape

(6865, 2)

Check the average length of a news story

In [10]:
lens = [len(t.split()) for t in train['STORY'].tolist()]
sum(lens) / len(lens)

107.65506190823015

100 seems to be a reasonable choice of max_seq_length

In [19]:
max_seq_length = 100

In [11]:
train_text = train['STORY'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

dev_text = dev['STORY'].tolist()
dev_text = np.array(dev_text, dtype=object)[:, np.newaxis]

In [22]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(bert_path)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Convert data to InputExample format

In [23]:
train_examples = convert_text_to_examples(train_text)
dev_examples = convert_text_to_examples(dev_text)

Convert data to BERT input format

In [24]:
(train_input_ids,
train_input_masks,
train_segment_ids) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(dev_input_ids,
dev_input_masks,
dev_segment_ids) = convert_examples_to_features(tokenizer, dev_examples, max_seq_length=max_seq_length)

Converting examples to features: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 763/763 [00:02<00:00, 341.35it/s]


In [25]:
train_input_ids.shape

(6865, 100)

Get the column with labels.
Get the number of unique labels - will need it to build the model.

In [12]:
num_labels = len(set(train['SECTION'].tolist()))
train_labels = train['SECTION'].to_numpy()
dev_labels = dev['SECTION'].to_numpy()
num_labels

4

EarlyStopping in case val_loss does not go down over 3 epochs

In [26]:
my_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=0,
                                patience=3,
                                verbose=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True)

Building the model: passing max_seq_length to define the input shape, and num_labels to define the number of units in the last layer.
Compiling the model: for multi-class classifiction where each label is a number, we use sparse_categorical_crossentropy loss and accuracy metric.

In [ ]:
model = build_model(max_seq_length, num_labels)
model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-5, beta_1=0.8),
                 metrics=["accuracy"])model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"])
model.summary()

In [ ]:
initialize_vars(sess)

In [31]:
epochs = 20
batch_size = 32

history = model.fit(
    [train_input_ids, train_input_masks, train_segment_ids],
    train_labels,
    validation_data=(
        [dev_input_ids, dev_input_masks, dev_segment_ids],
        dev_labels,
    ),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[my_callback]
)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 6865 samples, validate on 763 samples
Epoch 1/20
6865/6865 [==============================] - 1461s 213ms/sample - loss: 0.1480 - acc: 0.9511 - val_loss: 0.0714 - val_acc: 0.9751
Epoch 2/20
6865/6865 [==============================] - 1461s 213ms/sample - loss: 0.0470 - acc: 0.9849 - val_loss: 0.0599 - val_acc: 0.9790
Epoch 3/20
6865/6865 [==============================] - 1463s 213ms/sample - loss: 0.0216 - acc: 0.9926 - val_loss: 0.0525 - val_acc: 0.9830
Epoch 4/20
6865/6865 [==============================] - 1425s 208ms/sample - loss: 0.0118 - acc: 0.9956 - val_loss: 0.0629 - val_acc: 0.9790
Epoch 5/20
6865/6865 [==============================] - 1419s 207ms/sample - loss: 0.0080 - acc: 0.9964 - val_loss: 0.1230 - val_acc: 0.9712
Epoch 6/20
6865/6865 [==============================] - 1471s 214ms/sample - loss: 0.0138 - acc: 0.9943 - val_loss: 0.0380 - val_acc: 0.9882
Epoch 7/20
6865/6865 [==============================] - 1478s 215ms/sample - loss: 0.0073 - acc: 0.9964 - v

Plot model history for accuracy:

In [ ]:
plot_label(history, 'acc')

Plot model history for loss:

In [ ]:
plot_label(history, 'loss')

To save the model's weights:

In [ ]:
model.save_weights(r'weights')

 Load test data and convert to BERT input format

In [27]:
test = pd.read_excel(r'Data_Test.xlsx')
test_text = test['STORY'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

In [28]:
test_examples = convert_text_to_examples(test_text)

In [29]:
(test_input_ids,
test_input_masks,
test_segment_ids) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

Converting examples to features: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2748/2748 [00:07<00:00, 349.45it/s]


Let's predict test data labels now

In [32]:
pred = model.predict([test_input_ids,
    test_input_masks,
    test_segment_ids], batch_size = batch_size)

Convert probabilities to class labels

In [ ]:
pred_class = np.argmax(pred, axis = 1)

In [ ]:
To save predictions as a csv:

In [36]:
results = pd.DataFrame(data=pred_class, columns=['SECTION'])
results.to_csv(r'predictions.csv', index = False)